In [ ]:
# --- INSTALACIÓN Y CARGA ---
import pandas as pd
import numpy as np

# 1) Carga
RUTA = "SECOP_II_-_Procesos_de_Contratación_20250829.csv"

# Lectura robusta (manejo de separador y encoding)
df = pd.read_csv(
    RUTA,
    sep=",",
    encoding="utf-8",
    engine='python', # Added engine='python' to handle potential parsing issues
    on_bad_lines='skip' # Skip malformed lines
)

# 2) Vista general
print("Filas, Columnas:", df.shape)
display(df.head(3))
display(df.sample(3, random_state=7))
df.info()

# 3) Tipos de variables
tipos = df.dtypes.value_counts()
print("\nTipos de datos:\n", tipos)

# 4) Estandarizar nombres de columnas
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r"[^a-z0-9_]+", "_", regex=True)
      .str.replace(r"_+", "_", regex=True)
      .str.strip("_")
)

# 5) Identificación de columnas clave
posibles_ids = [c for c in df.columns if "id" in c or "uid" in c]
posibles_fechas = [c for c in df.columns if "fecha" in c or "date" in c]
posibles_valores = [c for c in df.columns if "valor" in c or "monto" in c or "cuantia" in c]
posibles_texto = [c for c in df.columns if df[c].dtype=="object"]

print("\nIDs:", posibles_ids)
print("Fechas:", posibles_fechas[:10])
print("Valores:", posibles_valores[:10])

# 6) Parseo de fechas más comunes
for col in posibles_fechas:
    try:
        df[col] = pd.to_datetime(df[col], errors="coerce", utc=False, dayfirst=True)
    except Exception:
        pass

# 7) Normalizar campos monetarios
def to_num(s):
    return (
        s.astype(str)
         .str.replace(r"[^\d\-\.\,]", "", regex=True)
         .str.replace(".", "", regex=True)       # quita separador miles punto
         .str.replace(",", ".", regex=False)     # coma -> punto decimal
    )

for col in posibles_valores:
    try:
        df[col] = pd.to_numeric(to_num(df[col]), errors="coerce")
    except Exception:
        pass

# 8) Limpieza de texto básico
for col in posibles_texto:
    df[col] = df[col].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)

# 9) Duplicados (usar columna ID si existe; si no, por combinación razonable)
id_cols = [c for c in posibles_ids if "contrato" in c or "proceso" in c] or posibles_ids
before = len(df)
if id_cols:
    df = df.drop_duplicates(subset=id_cols, keep="first")
else:
    df = df.drop_duplicates(keep="first")
print(f"\nDuplicados removidos: {before - len(df)}")

# 10) Valores nulos: reporte rápido
nulos = df.isna().mean().sort_values(ascending=False)
print("\n% NULOS (top 15):")
print((nulos*100).round(2).head(15))

# 11) Reglas mínimas para nulos en columnas críticas
cols_minimas = []
cols_minimas += id_cols[:1]
cols_minimas += [c for c in posibles_valores[:1]]
cols_minimas += [c for c in posibles_fechas[:1]]

for c in cols_minimas:
    if c in df.columns:
        df = df[df[c].notna()]

# 12) Distribuciones iniciales (frecuencias y describe)
cat_cols = [c for c in df.columns if df[c].dtype=="object"]
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
print("\nTop 10 categorías por columna (muestra):")
for c in cat_cols[:5]:
    print(f"\n{c}")
    print(df[c].value_counts(dropna=False).head(10))

print("\nEstadísticos numéricos:")
display(df[num_cols].describe().T)

# 13) Guardar dataset depurado
df.to_csv("SECOPII_contratos_limpio.csv", index=False)
print("\nArchivo guardado: SECOPII_contratos_limpio.csv")

Filas, Columnas: (110003, 59)


,Entidad,Nit Entidad,Departamento Entidad,Ciudad Entidad,OrdenEntidad,Entidad Centralizada,ID del Proceso,Referencia del Proceso,PCI,ID del Portafolio,...,Nombre del Proveedor Adjudicado,NIT del Proveedor Adjudicado,Codigo Principal de Categoria,Estado de Apertura del Proceso,Tipo de Contrato,Subtipo de Contrato,Categorias Adicionales,URLProceso,Codigo Entidad,Estado Resumen
0,ALCALDIA DE PASTO,891280000,Nariño,Pasto,Territorial,Descentralizada,CO1.REQ.4030898,CD-2023-1223,700694094,CO1.BDOS.3935438,...,INSTITUTO MUNICIPAL PARA LA RECREACION Y EL DE...,814000385,V1.90141700,Abierto,Otro,No Definido,No definido,https://community.secop.gov.co/Public/Tenderin...,"700,694,094",Adjudicado
1,INSTITUCION EDUCATIVA JUAN PABLO II.*,805028735,Valle del Cauca,Cali,Nacional,Descentralizada,CO1.REQ.7078722,4143.050.26.035.2024,718583263,CO1.BDOS.6946947,...,No Definido,No Definido,V1.43232300,Abierto,Prestación de servicios,No Definido,No definido,https://community.secop.gov.co/Public/Tenderin...,"718,583,263",Presentación de oferta
2,ALCALDIA LOCAL DE SUMAPAZ,899999061,Distrito Capital de Bogotá,Bogotá,Territorial,Centralizada,CO1.REQ.848621,FDLS-LP-098-2019,702096124,CO1.BDOS.823726,...,No Definido,No Definido,V1.81101500,Cerrado,Obra,No Definido,"V172101500, V172103300, V172141000, V195111600",https://community.secop.gov.co/Public/Tenderin...,"702,096,124",Presentación de observaciones


,Entidad,Nit Entidad,Departamento Entidad,Ciudad Entidad,OrdenEntidad,Entidad Centralizada,ID del Proceso,Referencia del Proceso,PCI,ID del Portafolio,...,Nombre del Proveedor Adjudicado,NIT del Proveedor Adjudicado,Codigo Principal de Categoria,Estado de Apertura del Proceso,Tipo de Contrato,Subtipo de Contrato,Categorias Adicionales,URLProceso,Codigo Entidad,Estado Resumen
26994,CONTRALORIA DISTRITAL DE CARTAGENA DE INDIAS,800194000,Bolívar,Cartagena,Territorial,Centralizada,CO1.REQ.2760467,CDC-003-2022,703161174,CO1.BDOS.2685175,...,Carlos Perinan,1143392645,V1.80111600,Abierto,Prestación de servicios,No Definido,No definido,https://community.secop.gov.co/Public/Tenderin...,"703,161,174",Adjudicado
25706,CACOM-2,800141627,Meta,Villavicencio,Nacional,Centralizada,CO1.REQ.6055055,022-00-E-CACOM-2-GRUTE-2024,701993016,CO1.BDOS.5935736,...,No Definido,No Definido,V1.56121802,Cerrado,Suministros,No Definido,No definido,https://community.secop.gov.co/Public/Tenderin...,"701,993,016",Presentación de observaciones
57889,SANTIAGO DE CALI DISTRITO ESPECIAL - SECRETARI...,890399011,Valle del Cauca,Cali,Territorial,Centralizada,CO1.REQ.4537962,4112.020.32.1.202-2023,705320893,CO1.BDOS.4436346,...,Ricardo Pinzón Saavedra,1015397090,V1.80111600,Abierto,Prestación de servicios,No Definido,V180111501,https://community.secop.gov.co/Public/Tenderin...,"705,320,893",Adjudicado


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110003 entries, 0 to 110002
Data columns (total 59 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   Entidad                                                 110003 non-null  object 
 1   Nit Entidad                                             110003 non-null  object 
 2   Departamento Entidad                                    110003 non-null  object 
 3   Ciudad Entidad                                          110003 non-null  object 
 4   OrdenEntidad                                            110003 non-null  object 
 5   Entidad Centralizada                                    110003 non-null  object 
 6   ID del Proceso                                          110003 non-null  object 
 7   Referencia del Proceso                                  110003 non-null  object 
 8   PCI                     

/tmp/ipython-input-3386144920.py:51: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce", utc=False, dayfirst=True)
/tmp/ipython-input-3386144920.py:51: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce", utc=False, dayfirst=True)
/tmp/ipython-input-3386144920.py:51: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce", utc=False, dayfirst=True)
/tmp/ipython-input-3386144920.py:51: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(


Duplicados removidos: 4304

% NULOS (top 15):
fecha_de_publicacion_fase_planeacion_precalificacion    100.0
nit_entidad                                               0.0
entidad                                                   0.0
ciudad_entidad                                            0.0
ordenentidad                                              0.0
entidad_centralizada                                      0.0
id_del_proceso                                            0.0
referencia_del_proceso                                    0.0
pci                                                       0.0
id_del_portafolio                                         0.0
departamento_entidad                                      0.0
nombre_del_procedimiento                                  0.0
descripci_n_del_procedimiento                             0.0
fecha_de_publicacion_del_proceso                          0.0
fase                                                      0.0
dtype: float64

Top 10 

,count,mean,std,min,25%,50%,75%,max
pci,105699.0,7.045828e+08,5.077058e+06,700000011.0,701675670.0,702809393.0,704683010.0,732568480.0
proveedores_con_invitacion_directa,105699.0,1.968420e-01,3.621335e+00,0.0,0.0,0.0,0.0,400.0
visualizaciones_del_procedimiento,105699.0,4.237334e+00,2.109474e+01,0.0,0.0,0.0,0.0,961.0
proveedores_que_manifestaron_interes,105699.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0
respuestas_al_procedimiento,105699.0,1.501433e-02,4.960932e-01,0.0,0.0,0.0,0.0,66.0
respuestas_externas,105699.0,1.986774e-04,2.554924e-02,0.0,0.0,0.0,0.0,6.0
conteo_de_respuestas_a_ofertas,105699.0,1.438046e-03,1.864839e-01,0.0,0.0,0.0,0.0,52.0
proveedores_unicos_con_respuestas,105699.0,5.129471e-01,3.999410e+00,0.0,0.0,0.0,0.0,531.0
numero_de_lotes,105699.0,8.005752e-01,6.024659e+00,0.0,0.0,0.0,0.0,244.0
id_estado_del_procedimiento,105699.0,6.343920e+01,1.168097e+01,0.0,50.0,70.0,70.0,100.0



Archivo guardado: SECOPII_contratos_limpio.csv
